<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/GTMBA/GTMBA_pirata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

## User inputs

In [ ]:
import glob as glob
import xarray as xr
import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
var = "sss"

time_start, time_end = ["2024-06-20", "2024-10-15"]
lat_start, lat_end = [0, 60]
lon_start, lon_end = [280, 360]

In [ ]:
dPath = "drive/MyDrive/datasets/GTMBA/download_17Jun2025/" + f"{var}/"
print(f"Reading data from:\n{dPath}")

## Read:
- `All` buoys.
- `Subset` those within the time of interest.
- `Subset` for the region of interest.

In [ ]:
fNames = glob.glob(dPath + "*.cdf")
fNames_subset = []

lons = []; lats = []
lons_subset = []; lats_subset = []

print(f"Data files in the region of interest and time:\n\n")
for fName in fNames:
  #print(fName)
  ds = xr.open_dataset(fName)
  lon, lat = [ds.lon.values[0], ds.lat.values[0]]
  #print(lon, lat)
  lons.append(lon)
  lats.append(lat)

  ds_subset = ds.sel(time=slice(time_start, time_end))
  nT = len(ds_subset.time)
  #print(nT)
  if nT > 0:
    lonx, latx = [ds_subset.lon.values[0], ds_subset.lat.values[0]]
    if (lonx >= lon_start) and (lonx <= lon_end) and (latx >= lat_start) and (latx <= lat_end):
      #print(lonx, latx)
      print(fName)
      fNames_subset.append(fName)
      lons_subset.append(lonx)
      lats_subset.append(latx)

  ds.close()

lons = np.asarray(lons)
lats = np.asarray(lats)

lons_subset = np.asarray(lons_subset)
lats_subset = np.asarray(lats_subset)

In [ ]:
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(color='k', alpha=0.5)
ax.add_feature(cfeature.LAND, facecolor='gray', alpha=0.1)

gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.4, color='k',
                  linestyle='--')
gl.top_labels = False
gl.right_labels = False

ax.set_extent([lon_start, lon_end, lat_start, lat_end], crs=ccrs.PlateCarree())
#ax.set_global()
# --

ax.plot(lons, lats, marker='o', ms=8, mfc='w', ls='None', transform=ccrs.PlateCarree())
ax.plot(lons_subset, lats_subset, marker='x', ms=8, mfc='b', ls='None', transform=ccrs.PlateCarree())

In [ ]:
if var == "sst":
  vName = 'T_25'
  qFlagName = 'QT_5025'
  qFlag = 2 # default quality https://www.pmel.noaa.gov/gtmba/data-quality-control

elif var == "sss":
  vName = 'S_41'
  qFlagName = 'QS_5041'
  qFlag = 2 # default quality https://www.pmel.noaa.gov/gtmba/data-quality-control

else:
  print(f"\n\n{var} has not yet been coded.\n\n")

In [ ]:
for iMoor, fName in enumerate(fNames_subset):
  ds = xr.open_dataset(fName)
  ds_subset = ds.sel(time=slice(time_start, time_end))
  good_data = ds_subset.where(ds[qFlagName]==qFlag)

  label = f"{ds_subset['lat'].values[0]:.1f}N, {360.-ds_subset['lon'].values[0]:.1f}W"
  #ds_subset[vName].plot(label=label, marker='x', ls='None')
  good_data[vName].plot(label=label, marker='o', ms=1, ls='None')

plt.legend(ncol=2)